# Analisador de empresas listadas no Guia Fácil

Tem como objetivo extrair informações relacionadas as empresas de uma determinada região e sua classificação no Google Meu Negócio, se houver.

```sql
CREATE TABLE empresa (
    id              VARCHAR(36)   NOT NULL,
    nome            VARCHAR(255)  NOT NULL,
    atividade       VARCHAR(255),
    pessoaJuridica  BOOLEAN,
    emailCliente    VARCHAR(255),
    cep             VARCHAR(20) ,
    cidade          VARCHAR(255),
    estado          VARCHAR(255),
    bairro          VARCHAR(255),
    uf              VARCHAR(5)  ,
    avaliacaoGoogle VARCHAR(255),
    PRIMARY KEY (id)
);

CREATE TABLE empresa_telefone (
    id                VARCHAR(36) NOT NULL,
    id_empresa        VARCHAR(36) NOT NULL,
    ddd               VARCHAR(5),
    telefone          VARCHAR(50) NOT NULL,
    telefoneCompleto  VARCHAR(100),
    whatsapp          BOOLEAN,
    telefone0800      BOOLEAN,
    PRIMARY KEY (id)
);

CREATE TABLE empresa_endereco (
    id         VARCHAR(36)  NOT NULL,
    id_empresa VARCHAR(36)  NOT NULL,
    endereco   VARCHAR(255) NOT NULL,
    PRIMARY KEY (id)
);
```

In [3]:
!pip install requests

In [4]:
!pip install psycopg2-binary

In [5]:
!pip install selenium
!pip install chromedriver-binary

In [6]:
DEBUG = True

def LOG(funcao, mensagem, pulaLinhaAntes = False, pulaLinhaDepois = False):
    if DEBUG == True:
        if pulaLinhaAntes:
            print('\n');
        print('[{}] => {}'.format(funcao, mensagem))
        if pulaLinhaDepois:
            print('\n');

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import chromedriver_binary
import json
import psycopg2
import uuid
import urllib.parse
import requests
import re
import time

In [8]:
def buscaAvaliacaoEmpresaNoGoogle(nomeEmpresa):

    LOG('buscaAvaliacaoEmpresaNoGoogle', 'Iniciado')

    site = 'https://www.google.com/search?q={}'.format(urllib.parse.quote(nomeEmpresa))

    LOG('buscaAvaliacaoEmpresaNoGoogle', 'Buscar empresa no google: {}'.format(site))

    avaliacao = ''
    try:
        options = Options()
        options.add_argument("--headless")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')
        options.add_argument('start-maximized')  
        options.add_argument('disable-infobars')
        options.add_argument("--disable-extensions")
        service = Service('/home/guiebitt/Workstation/Tools/Webdrivers/chromedriver')
        service.start()
        driver = webdriver.Remote(service.service_url, options = options)
        driver.get(site);
        element = driver.find_element(By.CSS_SELECTOR, ".Aq14fc")
        avaliacao = element.text
        driver.quit()
    except:
        avaliacao = ''

    LOG('buscaAvaliacaoEmpresaNoGoogle', 'Finalizado')

    return avaliacao

In [9]:
def existeEmpresaComNome(cursor, nome):

    LOG('existeEmpresaComNome', 'Iniciado')

    cursor.execute("SELECT 1 FROM empresa WHERE nome = '{}'".format(nome.replace("'", "''")))
    resultado = cursor.fetchone()
    existeEmpresa = False

    if (resultado != None):
        existeEmpresa = True

    LOG('existeEmpresaComNome', 'Existe empresa? {}'.format(existeEmpresa))
    LOG('existeEmpresaComNome', 'Finalizado')

    return existeEmpresa

In [10]:
def inserirTelefonesEmpresa(conexao, cursor, empresaId, telefones):
    
    LOG('inserirTelefonesEmpresa', 'Iniciado')

    for telefone in telefones:
        
        idTelefone = uuid.uuid4()
        
        LOG('inserirTelefonesEmpresa', 'UUID do registro: {}'.format(str(idTelefone)))
        
        whatsapp = False
        if telefone['whatsapp'] == 'true':
            whatsapp = True
        
        telefone0800 = False
        if telefone['telefone0800'] == 'true':
            telefone0800 = True

        cursor.execute("INSERT INTO empresa_telefone (id, id_empresa, ddd, telefone, telefoneCompleto, whatsapp, telefone0800) VALUES (%s, %s, %s, %s, %s, %s, %s)",(str(idTelefone), str(empresaId), telefone['ddd'], telefone['telefone'], telefone['telefoneCompleto'], whatsapp, telefone0800))
        conexao.commit()

    LOG('inserirTelefonesEmpresa', 'Finalizado')

    return

In [11]:
def inserirEnderecosEmpresa(conexao, cursor, empresaId, enderecos):

    LOG('inserirEnderecosEmpresa', 'Iniciado')

    for endereco in enderecos:
        
        idEndereco = uuid.uuid4()
        
        LOG('inserirEnderecosEmpresa', 'UUID do registro: {}'.format(str(idEndereco)))

        cursor.execute("INSERT INTO empresa_endereco (id, id_empresa, endereco) VALUES (%s, %s, %s)",(str(idEndereco), str(empresaId), endereco))
        conexao.commit()

    LOG('inserirEnderecosEmpresa', 'Finalizado')

    return

In [12]:
def inserirEmpresa(conexao, cursor, empresa):

    LOG('inserirEmpresa', 'Iniciado')

    pessoaJuridica = empresa['pessoaJuridica']
    if pessoaJuridica == False:
        LOG('inserirEmpresa', 'Pessoa fisica: {} nao inserida'.format(empresa['nome']))
        LOG('inserirEmpresa', 'Finalizado')
        return

    if (existeEmpresaComNome(cursor, empresa['nome']) == False):
        
        avaliacao = buscaAvaliacaoEmpresaNoGoogle('{}, {}'.format(empresa['nome'], empresa['cidade']))

        idEmpresa = uuid.uuid4()
        
        LOG('inserirEmpresa', 'UUID do registro: {}'.format(str(idEmpresa)))
        LOG('inserirEmpresa', 'Avaliação: {}'.format(avaliacao))
        
        cursor.execute("INSERT INTO empresa (id, nome, atividade, emailCliente, cep, cidade, estado, bairro, uf, pessoaJuridica, avaliacaoGoogle) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",(str(idEmpresa), empresa['nome'], empresa['atividade'], empresa['emailCliente'], empresa['cep'], empresa['cidade'], empresa['estado'], empresa['bairro'], empresa['uf'], pessoaJuridica, avaliacao))
        conexao.commit()

        inserirTelefonesEmpresa(conexao, cursor, idEmpresa, empresa['telefones'])
        inserirEnderecosEmpresa(conexao, cursor, idEmpresa, empresa['enderecos'])

        LOG('inserirEmpresa', 'Registro inserido: {}'.format(empresa['nome']))

    LOG('inserirEmpresa', 'Finalizado')

    return

In [13]:
def inserirEmpresas(empresas):

    LOG('inserirEmpresas', 'Iniciado')

    nomeBancoDados = 'COLETADEDADOS'
    usuarioBancoDados = 'postgres'
    senhaBancoDados = 'totvs@123'
    hostBancoDados = 'localhost'

    conexao = psycopg2.connect(dbname=nomeBancoDados, user=usuarioBancoDados, password=senhaBancoDados, host=hostBancoDados)
    cursor = conexao.cursor()

    LOG('inserirEmpresas', 'Conexao com banco de dados: {} efetuada'.format(nomeBancoDados))

    for empresa in empresas:
        inserirEmpresa(conexao, cursor, empresa)

    cursor.close()
    conexao.close()

    LOG('inserirEmpresas', 'Conexao com banco de dados: {} finalizada'.format(nomeBancoDados))
    LOG('inserirEmpresas', 'Finalizado')

    return

In [14]:
def buscarEmpresasDaPagina(categoria, cidade, estado, pagina):

    LOG('buscarEmpresasDaPagina', 'Iniciado')

    site = 'https://guiafacil.com/buscaAjax.php?busca={}&cidade={}&tipoBusca=segmento&estado={}&pagina={}'
    site = site.format(categoria, cidade, estado, pagina)

    LOG('buscarEmpresasDaPagina', 'Buscar: {}'.format(site))

    conteudo = urlopen(site).read()
    conteudoJson = json.loads(conteudo)
    resultado = conteudoJson['resultados']

    LOG('buscarEmpresasDaPagina', 'Quantidade de empresas da página: {}'.format(len(resultado)))
    LOG('buscarEmpresasDaPagina', 'Finalizado')

    return resultado

In [15]:
def buscarEmpresas(categoria, cidade, estado):

    LOG('buscarEmpresas', 'Iniciado')

    pagina = 1
    while(True):
        resultadoPagina = buscarEmpresasDaPagina(categoria, cidade, estado, pagina)
        if (len(resultadoPagina) == 0):
            break

        inserirEmpresas(resultadoPagina)
        pagina += 1

    LOG('buscarEmpresas', 'Finalizado')

    return

In [16]:
buscarEmpresas('', 'joinville', 'santa-catarina')

e] => Existe empresa? True
[existeEmpresaComNome] => Finalizado
[inserirEmpresa] => Finalizado
[inserirEmpresa] => Iniciado
[existeEmpresaComNome] => Iniciado
[existeEmpresaComNome] => Existe empresa? True
[existeEmpresaComNome] => Finalizado
[inserirEmpresa] => Finalizado
[inserirEmpresa] => Iniciado
[existeEmpresaComNome] => Iniciado
[existeEmpresaComNome] => Existe empresa? True
[existeEmpresaComNome] => Finalizado
[inserirEmpresa] => Finalizado
[inserirEmpresa] => Iniciado
[existeEmpresaComNome] => Iniciado
[existeEmpresaComNome] => Existe empresa? True
[existeEmpresaComNome] => Finalizado
[inserirEmpresa] => Finalizado
[inserirEmpresa] => Iniciado
[existeEmpresaComNome] => Iniciado
[existeEmpresaComNome] => Existe empresa? True
[existeEmpresaComNome] => Finalizado
[inserirEmpresa] => Finalizado
[inserirEmpresa] => Iniciado
[existeEmpresaComNome] => Iniciado
[existeEmpresaComNome] => Existe empresa? True
[existeEmpresaComNome] => Finalizado
[inserirEmpresa] => Finalizado
[inserirEm